In [7]:
library(DEqMS)
library(patchwork)
library(tidyverse)

source("../../evaluation_utils/evaluation/DE_analysis.R")
source("../../evaluation_utils/plots/DE_plots.R")
source("../../evaluation_utils/filtering/filtering_normalization.R")
source("../../evaluation_utils/plots/eda_plots.R")

# Bacterial

In [8]:
labs_list = c('lab_A', 'lab_B', 'lab_C', 'lab_D' , 'lab_E')
dataset <- "bacterial"
mode <- "balanced"

path_to_reports = paste0('/home/yuliya/repos/cosybio/FedProt/data/bacterial_data/', mode, '/')
out_path = paste0('/home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/', dataset, '/')

In [9]:
central_intensities = NULL
central_counts = NULL
central_batch_info = NULL

dir.create(paste0(out_path, 'plots'), recursive = TRUE)
dir.create(paste0(out_path, 'results'), recursive = TRUE)


for (name in labs_list) {
    batch_info = read_tsv(paste0(path_to_reports, name, '/metadata_short.tsv'), show_col_types = FALSE)
    intensities = read_tsv(paste0(path_to_reports, name, '/protein_groups_matrix.tsv'), show_col_types = FALSE)
    counts = read_tsv(paste0(path_to_reports, name, '/protein_counts.tsv'), show_col_types = FALSE)

    if(is.null(central_intensities)){
        central_intensities = intensities
        central_counts = counts
        central_batch_info = batch_info
    } else {
        central_intensities = full_join(central_intensities, intensities, by = 'rowname')
        central_counts = full_join(central_counts, counts, by = 'rowname')
        central_batch_info = rbind(central_batch_info, batch_info)
    }
}
central_batch_info <- central_batch_info %>%
    mutate(lab = as.factor(lab), 
    condition = as.factor(condition))

cat('\n\nDataset: ', dataset, "\n")
cat('\tNumber of proteins: ', nrow(central_intensities), '\n')
cat('\tNumber of samples: ', ncol(central_intensities)-1, '\n')

central_intensities <- central_intensities %>% column_to_rownames('rowname')
central_counts <- central_counts %>% column_to_rownames('rowname')

central_counts$count <- apply(central_counts, 1, min, na.rm = TRUE)
central_counts <- central_counts %>% select(count) %>% as.data.frame()

central_intensities <- central_intensities[, central_batch_info$file]
central_intensities <- filter_by_condition(central_intensities, central_batch_info, 
    'file', c('Glu', 'Pyr'), 'condition', min_f=0.8)
central_intensities <- filter_na_proteins(central_intensities, central_batch_info, "file")

cat("Rows after all filters:", nrow(central_intensities), "\n")

central_intensities <- log2(central_intensities + 1)

# plot before
plot_pca_before <- pca_plot(central_intensities, central_batch_info, 
    title=paste0("Bacterial data before correction"), 
    quantitative_col_name='file', col_col='condition', shape_col='lab')

# batch effects correction
design <- model.matrix(~ condition, data = central_batch_info)
pg_corrected <- removeBatchEffect(central_intensities, batch=central_batch_info$lab, design=design)

plot_pca_after <- pca_plot(pg_corrected, central_batch_info, 
    title=paste0("Bacterial data after correction (removeBatchEffect)"), 
    quantitative_col_name='file', col_col='condition', shape_col='lab')

layout <- (plot_pca_before | plot_pca_after)
ggsave(file = paste0(out_path, "plots/PCA_plots.svg"), plot = layout, width = 9.5, height = 6)

# DE analysis
design <- make_design(central_batch_info, 'condition')
contrasts <- makeContrasts(Glu-Pyr, levels = colnames(design))
de_results <- run_DE(pg_corrected, central_counts, design, contrasts)
de_results <- de_results %>% rownames_to_column('Protein')
write.table(
    de_results, 
    file = paste0(out_path, 'results/central_corrected_res.tsv'), 
    sep = "\t", quote = FALSE, row.names = FALSE
)


# # On uncorrected
# # DE analysis
# design <- make_design(central_batch_info, 'condition', "lab")
# contrasts <- makeContrasts(Glu-Pyr, levels = colnames(design))
# de_results <- run_DE(central_intensities, central_counts, design, contrasts)
# de_results <- de_results %>% rownames_to_column('Protein')
# write.table(
#     de_results, 
#     file = paste0(out_path, 'results/central_res.tsv'), 
#     sep = "\t", quote = FALSE, row.names = FALSE
# )

# plot volcano plot
plot_result <- volcano_plot(
    de_results, paste(dataset, "central", ", Glu/Pyr"),
    pval_threshold = 0.05, logfc_threshold = 0.5,
    show_names = FALSE
)
ggsave(file = paste0(out_path, 'plots/central_volcano_plot.svg'), plot = plot_result, width = 8, height = 5)


Warning message in dir.create(paste0(out_path, "plots"), recursive = TRUE):
“'/home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/bacterial/plots' already exists”
Warning message in dir.create(paste0(out_path, "results"), recursive = TRUE):
“'/home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/bacterial/results' already exists”




Dataset:  bacterial 
	Number of proteins:  3059 
	Number of samples:  118 
Filtering by condition - two not-NA per condition
	Before filtering: 3059 102 
	After filtering: 2831 102 
Filtering out features that have NAs in all columns
	Before filtering: 2831 102 
	After filtering: 2831 102 
Rows after all filters: 2831 


Warning message:
“Partial NA coefficients for 559 probe(s)”


In [10]:
system(
    "mkdir -p /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/bacterial/balanced/results"
)
system(
"python /home/yuliya/repos/cosybio/FedProt/evaluation_utils/fedprot_prototype/fedprot_script.py /home/yuliya/repos/cosybio/FedProt/data/bacterial_data/ balanced lab_A,lab_B,lab_C,lab_D,lab_E /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/bacterial/"
)
system(
    "cp /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/bacterial/balanced/results/* /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/bacterial/results/"
)
system(
    "rm -rf /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/bacterial/balanced/"
)

# TMT_data

In [11]:
labs_list = c('Center1', 'Center2', 'Center3')
dataset <- "TMT_data_SHARED"
mode <- "01_smaller_lib_balanced_PG_MajorPG_SHARED"

path_to_reports = paste0('/home/yuliya/repos/cosybio/FedProt/data/TMT_data/', mode, '/')
out_path = paste0('/home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/', dataset, '/')

In [12]:
dir.create(paste0(out_path, 'plots'), recursive = TRUE)
dir.create(paste0(out_path, 'results'), recursive = TRUE)

central_intensities = NULL
central_counts = NULL

central_batch_info = read_tsv(paste0(path_to_reports, 'metadata.tsv'), show_col_types = FALSE)
central_batch_info <- central_batch_info[central_batch_info$Group != "Common Reference" & central_batch_info$Group != "in_silico", ]
central_batch_info <- central_batch_info  %>%
    filter(Group != "Common Reference") %>%
    # rename Pool1 to Pool7 in the Pool column
    # mutate(Pool = ifelse(Pool == "Pool1", "Pool7", Pool)) %>%
    mutate(
        file = Quantitative.column.name,
        lab = as.factor(Center),
        condition = factor(Group, levels = c('heathy', 'FSGS')),
        Pool = as.factor(Pool)
    )
cat("Number of samples: ", nrow(central_batch_info), "\n")

for (name in labs_list) {
    intensities = read_tsv(paste0(path_to_reports, name, '/pg_intensities.tsv'), show_col_types = FALSE)
    counts = read_tsv(paste0(path_to_reports, name, '/pg_counts.tsv'), show_col_types = FALSE)

    if(is.null(central_intensities)){
        central_intensities = intensities
        central_counts = counts
    } else {
        central_intensities = full_join(central_intensities, intensities, by = 'Majority.protein.IDs')
        central_counts = full_join(central_counts, counts, by = 'Majority.protein.IDs')
    }
}

cat('\n\nDataset: ', dataset, "\n")
cat('\tNumber of proteins: ', nrow(central_intensities), '\n')
cat('\tNumber of samples: ', ncol(central_intensities)-1, '\n')

# transform and order
central_intensities <- central_intensities %>% column_to_rownames('Majority.protein.IDs')
central_counts <- central_counts %>% column_to_rownames('Majority.protein.IDs')

central_intensities <- central_intensities[, central_batch_info$file]

# select minimal count across column for each protein (with na.rm = TRUE)
central_counts$count <- apply(central_counts, 1, min, na.rm = TRUE)
central_counts <- central_counts %>% select(count)
# Filter - remove proteins with 1 count 
central_intensities <- central_intensities[rownames(central_intensities) %in% rownames(filter(central_counts, count > 1)), ]
cat("Rows after filtering 1 count:", nrow(central_intensities), "\n")

########################################################################################
# filter by condition
central_intensities <- filter_na_proteins(central_intensities, central_batch_info, "file")
central_intensities <- filter_by_condition(central_intensities, central_batch_info, 
        'file', c('heathy', 'FSGS'), 'condition', min_f=0.8)

cat("Rows after all filters:", nrow(central_intensities), "\n")

# NORMALIZATION for plots
# median normalization
central_intensities <- medianNorm(central_intensities) %>% as.data.frame()
central_intensities <- central_intensities[sort(rownames(central_intensities)), central_batch_info$file]

# IRS normalization
corrected_intensities <- NULL

for(center in labs_list){
    cat("IRS normalization for center: ", center, "\n")
    center_intensities <- central_intensities[, central_batch_info$lab == center]
    center_batch_info <- central_batch_info[central_batch_info$lab == center, ]
    cat("Shape of the data: ", dim(center_intensities), "\n")

    irs_res <- irsNorm_in_silico_single_center(
        center_intensities, center_batch_info,
        pool_col = "Pool",
        column_name = "file",
        center = name,
        aggregation_method = "average",
        add_refs = FALSE
    )
    # add to the central data
    if(is.null(corrected_intensities)){
        corrected_intensities = irs_res$corrected_data
    } else {
        corrected_intensities = cbind(corrected_intensities, irs_res$corrected_data)
    }
}

cat("Data after IRS normalization: ", dim(corrected_intensities), " rows\n")
central_intensities <- corrected_intensities[, central_batch_info$file]
central_intensities <- log2(central_intensities + 1)

central_batch_info <- central_batch_info %>% mutate(
    condition = factor(condition, levels = c('heathy', 'FSGS')),
    Pool = as.factor(Pool)
  )

# plot before
plot_pca_before <- pca_plot(central_intensities, central_batch_info, 
    title=paste0("Human plasma data before correction"), 
    quantitative_col_name='file', col_col='condition', shape_col='Pool')

# batch effects correction
design <- model.matrix(~condition, data = central_batch_info)
central_intensities <- central_intensities[, central_batch_info$file]
pg_corrected <- removeBatchEffect(central_intensities, batch=central_batch_info$Pool, design=design) %>% as.data.frame()
# pg_corrected <- removeBatchEffect(central_intensities, batch=central_batch_info$Pool) %>% as.data.frame()

cat("Data after correction: ", dim(pg_corrected), " rows\n")
plot_pca_after <- pca_plot(pg_corrected, central_batch_info, 
    title=paste0("Human plasma data after (removeBatchEffect)"), 
    quantitative_col_name='file', col_col='condition', shape_col='Pool')

layout <- (plot_pca_before | plot_pca_after)
ggsave(file = paste0(out_path, "plots/PCA_plots.svg"), plot = layout, width = 9.5, height = 6)


# DE analysis
design <- make_design(central_batch_info, 'condition')
contrasts <- makeContrasts(heathy-FSGS, levels = colnames(design))
de_results <- run_DE(pg_corrected, central_counts, design, contrasts)
de_results <- de_results %>% rownames_to_column('Protein')
write.table(
    de_results, 
    file = paste0(out_path, 'results/central_corrected_res.tsv'), 
    sep = "\t", quote = FALSE, row.names = FALSE
)

# # (central uncorrected)
# # DE analysis
# design <- make_design(central_batch_info, 'condition', "Pool")
# contrasts <- makeContrasts(heathy-FSGS, levels = colnames(design))
# de_results <- run_DE(central_intensities, central_counts, design, contrasts)
# de_results <- de_results %>% rownames_to_column('Protein')
# write.table(
#     de_results, 
#     file = paste0(out_path, 'results/central_res.tsv'), 
#     sep = "\t", quote = FALSE, row.names = FALSE
# )
   
   

# plot volcano plot
plot_result <- volcano_plot(
    de_results, paste(dataset, "central", ", healthy/FSGS"),
    pval_threshold = 0.05, logfc_threshold = 0.25,
    show_names = FALSE
)
ggsave(file = paste0(out_path, 'plots/central_volcano_plot.svg'), plot = plot_result, width = 8.5, height = 5)


Warning message in dir.create(paste0(out_path, "plots"), recursive = TRUE):
“'/home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/TMT_data_SHARED/plots' already exists”
Warning message in dir.create(paste0(out_path, "results"), recursive = TRUE):
“'/home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/TMT_data_SHARED/results' already exists”


Number of samples:  59 


Dataset:  TMT_data_SHARED 
	Number of proteins:  628 
	Number of samples:  65 
Rows after filtering 1 count: 583 
Filtering out features that have NAs in all columns
	Before filtering: 583 59 
	After filtering: 580 59 
Filtering by condition - two not-NA per condition
	Before filtering: 580 59 
	After filtering: 532 59 
Rows after all filters: 532 
IRS normalization for center:  Center1 
Shape of the data:  532 20 
Shape of intensities for pool  Pool1 : 532 10 
Shape of intensities for pool  Pool2 : 532 10 
IRS normalization for center:  Center2 
Shape of the data:  532 19 
Shape of intensities for pool  Pool3 : 532 9 
Shape of intensities for pool  Pool5 : 532 10 
IRS normalization for center:  Center3 
Shape of the data:  532 20 
Shape of intensities for pool  Pool4 : 532 10 
Shape of intensities for pool  Pool6 : 532 10 
Data after IRS normalization:  532 59  rows


Warning message:
“Partial NA coefficients for 209 probe(s)”


Data after correction:  532 59  rows


In [13]:
system(
    "mkdir -p /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/TMT_data_SHARED/01_smaller_lib_balanced_PG_MajorPG_SHARED/results"
)
system(
"python /home/yuliya/repos/cosybio/FedProt/evaluation_utils/fedprot_prototype/fedprot_script.py /home/yuliya/repos/cosybio/FedProt/data/TMT_data/  01_smaller_lib_balanced_PG_MajorPG_SHARED Center1,Center2,Center3 /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/TMT_data_SHARED/"
)
system(
    "cp /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/TMT_data_SHARED/01_smaller_lib_balanced_PG_MajorPG_SHARED/results/* /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/TMT_data_SHARED/results/"
)
system(
    "rm -rf /home/yuliya/repos/cosybio/FedProt/evaluation/batch_effects_eval/TMT_data_SHARED/01_smaller_lib_balanced_PG_MajorPG_SHARED/"
)